### Use this Jupyter Notebook as a guide to run your trained model in inference mode

created by Anton Morgunov

inspired by [tensorflow object detection API tutorial](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html#exporting-a-trained-model)

Modified by Mathias Ramm Haugland

Your first step is going to specify which unit you are going to work with for inference. Select between GPU or CPU and follow the below instructions for implementation.

### EVAL

##### Run "inference_as_raw_output" on evaluation data
##### Use result in "pr_curve" to make a pr_curve
##### Choose a confidence threshold from the curve
##### Use this as input to "inference_with_plot" along with test/eval data

### TEST
##### Run inference_as_raw_output on test data
##### Use result in test_pr to get precision recall with conf thresh from eval
##### Use this thresh as input to "inference_with_plot" 

In [ ]:
import os # importing OS in order to make GPU visible
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" # do not change anything in here

# specify which device you want to work on.
# Use "-1" to work on a CPU. Default value "0" stands for the 1st GPU that will be used
os.environ["CUDA_VISIBLE_DEVICES"]="0" # TODO: specify your computational device

In [ ]:
import tensorflow as tf # import tensorflow

# checking that GPU is found
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

In [ ]:
# other import
import numpy as np
from PIL import Image
%matplotlib inline
from matplotlib import pyplot as plt
from tqdm import tqdm

Next you will import import scripts that were already provided by Tensorflow API. **Make sure that Tensorflow is your current working directory.**

In [ ]:
import sys # importyng sys in order to access scripts located in a different folder

path2scripts = '/home/hemin/mathiasrammhaugland/master/Tensorflow/models/research/'
sys.path.insert(0, path2scripts) # making scripts in models/research available for import

In [ ]:
# importing all scripts that will be needed to export your model and use it for inference
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

Now you can import and build your trained model:

In [ ]:
# NOTE: your current working directory should be Tensorflow.

# TODO: specify two pathes: to the pipeline.config file and to the folder with trained model.
path2config ='/home/hemin/mathiasrammhaugland/master/Tensorflow/workspace/exported_models/efficientdet_d0/classification_snbi4_4/pipeline.config'
path2model = '/home/hemin/mathiasrammhaugland/master/Tensorflow/workspace/exported_models/efficientdet_d0/classification_snbi4_4/checkpoint/'

In [ ]:
# do not change anything in this cell
configs = config_util.get_configs_from_pipeline_file(path2config) # importing config
model_config = configs['model'] # recreating model config
detection_model = model_builder.build(model_config=model_config, is_training=False) # importing model

In [ ]:
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(path2model, 'ckpt-0')).expect_partial()

Next, path to label map should be provided. Category index will be created based on labal map file

In [ ]:
path2label_map = '/home/hemin/mathiasrammhaugland/master/Tensorflow/workspace/data/classification/label_map.pbtxt' # TODO: provide a path to the label map file
category_index = label_map_util.create_category_index_from_labelmap(path2label_map,use_display_name=True)

Now, a few supporting functions will be defined

In [ ]:
def detect_fn(image):
    """
    Detect objects in image.
    
    Args:
      image: (tf.tensor): 4D input image
      
    Returs:
      detections (dict): predictions that model made
    """

    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)

    return detections

In [ ]:
def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      numpy array with shape (img_height, img_width, 3)
    """
    
    return np.array(Image.open(path))

In [ ]:
import json
def make_gt_im(im_np,gt_dic,cat_idx):
    height = int(gt_dic['height'])
    width = int(gt_dic['width'])
    
    boxes = []
    classes = []
    scores = []
    if height != im_np.shape[0] or width != im_np.shape[1]:
        print("BBox hasnt same dimensions as image!")
    else:
        for box in gt_dic['bbox']:
            coordinates = [int(box['ymin'])/height,int(box['xmin'])/width,int(box['ymax'])/height,int(box['xmax'])/width]
            boxes.append(coordinates)
            
            if (box['label'] == 'polyp'):
                classes.append(1)
            if (box['label'] == 'Hyperplasia'):
                classes.append(1)
            elif (box['label'] == 'Adenoma'):
                classes.append(2)
                
            scores.append(1)
        boxes = np.array(boxes)
        viz_utils.visualize_boxes_and_labels_on_image_array(
                im_np,
                boxes,
                classes,
                scores,
                cat_idx,
                use_normalized_coordinates=True,
                max_boxes_to_draw=100,
                min_score_thresh=0.0,
                agnostic_mode=False,
                line_thickness=4)
        
        return im_np
    #return im_np_with_gt

**Next function is the one that you can use to run inference and plot results an an input image:**

In [ ]:
def inference_with_plot(path2images, outpath, box_th=0.05, gt_json = None, nms_th=0.2):
    """
    Function that performs inference and plots resulting b-boxes
    
    Args:
      path2images: an array with pathes to images
      box_th: (float) value that defines threshold for model prediction.
      gt_json: json file with gt_boxes to be plotted on the same image.
      
    Returns:
      None
    """
    
    if gt_json:
        f = open(gt_json)
        gt_dic = json.load(f)
    
    i=0
    for image_path in path2images:
        
        print('Running inference for {}... '.format(image_path), end='')
        
        im_name = image_path.split('/')[-1].replace('jpg','jpg')
        image_np = load_image_into_numpy_array(image_path)

        input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
        detections = detect_fn(input_tensor)
        # All outputs are batches tensors.
        # Convert to numpy arrays, and take index [0] to remove the batch dimension.
        # We're only interested in the first num_detections.
        num_detections = int(detections.pop('num_detections'))
        detections = {key: value[0, :num_detections].numpy()
                      for key, value in detections.items()}
        
        detections['num_detections'] = num_detections

        # detection_classes should be ints.
        detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
        label_id_offset = 1
        image_np_with_detections = image_np.copy()
        
        
        if nms_th: # filtering rectangles if nms threshold was passed in as a parameter
        # creating a zip object that will contain model output info as
            output_info = list(zip(detections['detection_boxes'],
                                    detections['detection_scores'],
                                    detections['detection_classes']
                                        )
                                )
            #print(nms_th)
            boxes, scores, classes = nms(output_info, thd = nms_th)
        
        boxes = np.array(boxes)
        scores = np.array(scores)
        classes = np.array(classes)
        viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                boxes,
                classes+label_id_offset,
                scores,
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=6,
                min_score_thresh=box_th,
                agnostic_mode=False,
                line_thickness=4)
        if gt_json:
            image_np_with_gt = make_gt_im(image_np.copy(),gt_dic[im_name],category_index)
            im_comb = np.hstack([image_np_with_detections,image_np_with_gt])
            image_np_with_detections = im_comb
        plt.figure(figsize=(15,10))
        plt.imshow(image_np_with_detections)
        plt.savefig(outpath+im_name)
        i+=1
        print('Done')
    #plt.show()
    if gt_json:
        f.close()

In [ ]:
#Print images with detection vs bboxes:
#get test images on array format2
t = 'test'
v= 'v24'
path = '/home/hemin/mathiasrammhaugland/master/set3/SNBI4/images_snbi4/' #eval or test
image_path_array = []
for file_name in os.listdir(path):
    image_path_array.append(path+file_name)


inference_with_plot(image_path_array,
                    outpath = f"./{t}_results_images/classification_snbi4_4/", #test or eval data?
                    box_th = 0.22,
                    gt_json = f'/home/hemin/mathiasrammhaugland/master/set3/set3_class.json',#WLI normally
                    nms_th=0.2
                   )
                 

Next, we will define a few other supporting functions:

In [ ]:
def nms(rects, thd=0.2):
    """
    Filter rectangles
    rects is array of oblects ([x1,y1,x2,y2], confidence, class)
    thd - intersection threshold (intersection divides min square of rectange)
    """
    out = []

    remove = [False] * len(rects)

    for i in range(0, len(rects) - 1):
        if remove[i]:
            continue
        inter = [0.0] * len(rects)
        for j in range(i, len(rects)):
            if remove[j]:
                continue
            inter[j] = intersection(rects[i][0], rects[j][0]) / min(square(rects[i][0]), square(rects[j][0]))

        max_prob = 0.0
        max_idx = 0
        for k in range(i, len(rects)):
            if inter[k] >= thd:
                if rects[k][1] > max_prob:
                    max_prob = rects[k][1]
                    max_idx = k

        for k in range(i, len(rects)):
            if (inter[k] >= thd) & (k != max_idx):
                remove[k] = True

    for k in range(0, len(rects)):
        if not remove[k]:
            out.append(rects[k])

    boxes = [box[0] for box in out]
    scores = [score[1] for score in out]
    classes = [cls[2] for cls in out]
    return boxes, scores, classes


def intersection(rect1, rect2):
    """
    Calculates square of intersection of two rectangles
    rect: list with coords of top-right and left-boom corners [x1,y1,x2,y2]
    return: square of intersection
    """
    x_overlap = max(0, min(rect1[2], rect2[2]) - max(rect1[0], rect2[0]));
    y_overlap = max(0, min(rect1[3], rect2[3]) - max(rect1[1], rect2[1]));
    overlapArea = x_overlap * y_overlap;
    return overlapArea


def square(rect):
    """
    Calculates square of rectangle
    """
    return abs(rect[2] - rect[0]) * abs(rect[3] - rect[1])

**Next function is the one that you can use to run inference and save results into a file:**

In [ ]:
def inference_as_raw_output(path2images,
                            box_th = 0.0,
                            nms_th = 0.2,
                            to_file = False,
                            data = None,
                            path2dir = True):
    """
    Function that performs inference and return filtered predictions
    
    Args:
      path2images: an array with pathes to images
      box_th: (float) value that defines threshold for model prediction. Consider 0.25 as a value.
      nms_th: (float) value that defines threshold for non-maximum suppression. Consider 0.5 as a value.
      to_file: (boolean). When passed as True => results are saved into a file. Writing format is
      path2image + (x1abs, y1abs, x2abs, y2abs, score, conf) for box in boxes
      data: (str) name of the dataset you passed in (e.g. test/validation)
      path2dir: (str). Should be passed if path2images has only basenames. If full pathes provided => set False.
      
    Returs:
      detections (dict): filtered predicclassificationtions that model made
    """
        
    print (f'Current data set is {data}')
    print (f'Ready to start inference on {len(path2images)} images!')
    
    for image_path in tqdm(os.listdir(path2images)):
        #print(image_path)
        if path2dir: # if a path to a directory where images are stored was passed in
            image_path = os.path.join(path2images, image_path.strip())

        image_np = load_image_into_numpy_array(image_path)

        input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
        detections = detect_fn(input_tensor)
        
        # checking how many detections we got
        num_detections = int(detections.pop('num_detections'))
        
        # filtering out detection in order to get only the one that are indeed detections
        detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
        
        # detection_classes should be ints.
        detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
        
        # defining what we need from the resulting detection dict that we got from model output
        key_of_interest = ['detection_classes', 'detection_boxes', 'detection_scores']
        
        # filtering out detection dict in order to get only boxes, classes and scores
        detections = {key: value for key, value in detections.items() if key in key_of_interest}
        
        if box_th: # filtering detection if a confidence threshold for boxes was given as a parameter
            for key in key_of_interest:
                scores = detections['detection_scores']
                current_array = detections[key]
                filtered_current_array = current_array[scores > box_th]
                detections[key] = filtered_current_array
        
        if nms_th: # filtering rectangles if nms threshold was passed in as a parameter
            # creating a zip object that will contain model output info as
            output_info = list(zip(detections['detection_boxes'],
                                   detections['detection_scores'],
                                   detections['detection_classes']
                                  )
                              )
            boxes, scores, classes = nms(output_info)
            
            detections['detection_boxes'] = boxes # format: [y1, x1, y2, x2]
            detections['detection_scores'] = scores
            detections['detection_classes'] = classes

        #print(detections)

        if to_file and data: # if saving to txt file was requested

            image_h, image_w, _ = image_np.shape
            file_name = f'{data}_results.txt'
            
            line2write = list()
            line2write.append(os.path.basename(image_path))
            
            with open(file_name, 'a+') as text_file:
                # iterating over boxes
                for b, s, c in zip(boxes, scores, classes):
                    
                    y1abs, x1abs = b[0] * image_h, b[1] * image_w
                    y2abs, x2abs = b[2] * image_h, b[3] * image_w
                    
                    list2append = [x1abs, y1abs, x2abs, y2abs, s, c]
                    line2append = ','.join([str(item) for item in list2append])
                    
                    line2write.append(line2append)
                
                line2write = ' '.join(line2write)
                text_file.write(line2write + os.linesep)
    return detections

Use the inference_as_raw_output function to get these x and y coordinates (and classification), and compare these with values from a json file of the test data. 

In [ ]:
#test function to write each video to a txt file
#test_folders="/media/hemin/Data/Hemin'sFiles/My_Dataset/OUS-NBI-ColonVDB-master/test/WLI/polyps/" #WLI or 'NBI/'
t= "eval"

test_folders="/home/hemin/mathiasrammhaugland/master/set2/SNBI4/"+t #SNBI "test/all or eval"

#For multiple test folders:
#for folder in os.listdir(test_folders):
#    inference_as_raw_output(test_folders+'/'+folder+"/", to_file=True, data="test_results/detection_wli_1/"+folder+"_wli")

#Only one test folder:
#inference_as_raw_output(test_folders, to_file=True, data='test_results/classification_snbi4_4/class_snbi4_'+t)

#ex3
inference_as_raw_output("/home/hemin/mathiasrammhaugland/master/set3/SNBI4/Adenoma", 
                        to_file=True,
                        data='test_results/classification_snbi4_4/class_snbi4_test_kumc_adenoma')

In [ ]:
import cv2
#calculate IoU between one gt and one pred box, given a confidence threshold:
def calc_iou(pred_box,gt_box,im_h,im_w):
    empty = np.zeros(int(im_h)*int(im_w)).reshape([int(im_h),int(im_w)])
    actual = empty.copy()

    actual = cv2.rectangle(actual,[gt_box[0],gt_box[1]],[gt_box[2],gt_box[3]],1,-1)
    
    pred = empty.copy()
    
    pred = cv2.rectangle(pred,[pred_box[0],pred_box[1]],[pred_box[2],pred_box[3]],2,-1)

    combined = np.squeeze(np.asarray(np.add(pred, actual)))
    unique, counts = np.unique(combined, return_counts=True) #unique=index, counts=#

    zipped = dict(zip(unique,counts))

    for un in [1.0,2.0,3.0]:
        if un not in zipped:
            zipped[un] = 0.0
            
            
    
            
    #0.0 = true negative
    #1.0 = false negative
    #2.0 = false positive
    #3.0 = true positive/intersection
    #union = 1+2+3

    intersection = zipped.get(3.0)

    union = zipped.get(1.0) + zipped.get(2.0) + zipped.get(3.0)

    iou_i = intersection/union
    #print(iou_i)
    return iou_i

In [ ]:
def pos_and_neg_pr_im(pred_mat, gt_mat, im_h, im_w, thresh, gt_class, pred_class_scores,iou_th=0.2):
    tp = 0
    tp_fp = 0
    tp_fn = 0    

    i = 0

    for pred_box in pred_mat:
        conf_score = pred_class_scores[i][1]
        class_pred = pred_class_scores[i][0]
        iou_j = -1 #the highest iou
        if conf_score >= thresh:
            for gt_box in gt_mat:
                class_gt = gt_class[0]
                iou_i = calc_iou(pred_box,gt_box,im_h,im_w)


                if iou_i > iou_j:
                    if class_pred == class_gt:
                        iou_j = iou_i
                            
            tp_fp = tp_fp+1
            if iou_j>=iou_th: #our IOU threshold
                tp = tp+1
        i = i+1
                    
    
    tp_fn = tp_fn + len(gt_mat)
    
    
    return tp,tp_fp, tp_fn
        

In [ ]:
#different metrics
def metrics(pred_lst, gt_lst, gt_class_lst, pred_class_score_lst, h_lst, w_lst, iou_th = 0.2, thresh = 0.2):
    #get pr and rc
    tp_sum = 0
    tp_fp_sum = 0
    tp_fn_sum = 0
    
    for i in range(len(pred_lst)):
        tp, tp_fp, tp_fn = pos_and_neg_pr_im(pred_lst[i],
                                             gt_lst[i],
                                             h_lst[i],
                                             w_lst[i],
                                             thresh, 
                                             gt_class_lst[i],
                                             pred_class_score_lst[i],
                                             iou_th)
        tp_sum += tp
        tp_fp_sum += tp_fp
        tp_fn_sum += tp_fn


    print("Conf Thresh: {}".format(thresh))
    print("TPs: {}".format(tp_sum))
    print("TPFPs: {}".format(tp_fp_sum))
    print("TPFNs: {}".format(tp_fn_sum))
    
    if tp_fp_sum == 0:
        precision = 1
        recall = tp_sum/tp_fn_sum
    elif tp_fn_sum == 0:
        precision = tp_sum/tp_fp_sum
        recall = 1
    else:
        precision = tp_sum/(tp_fp_sum)
        recall = tp_sum/(tp_fn_sum)
    print("Prec: {}".format(precision))
    print("Recall: {}".format(recall))
    return precision, recall, tp_sum,tp_fp_sum, tp_fn_sum
        

In [ ]:
import json
#FOR CALCULATING PR CURVE OF EVALUATION DATA TO FIND BEST CONF THRESHOLD

t = 'test'
mo = 'snbi4'

def data_prep(vvv=None):
    #gt_file = f'/home/hemin/mathiasrammhaugland/master/set2/gt_boxes/class_{t}_piccolo_wli.json' #wli or nbi
    gt_file = '/home/hemin/mathiasrammhaugland/master/set3/set3_class.json'
    json_file = open(gt_file)
    gt = json.load(json_file)
    #pred_file = open(f'./test_results/classification_{mo}_4/class_{mo}_{t}_results.txt','r') #only "wli" here when testin wli
    pred_file = open(f'./test_results/classification_{mo}_4/class_{mo}_{t}_kumc_all_results.txt','r')
    gt_lst= [] #list of all bboxes from one video.
    pred_lst = [] #list of all prediction bboxes from the same video.
    w_lst, h_lst = [],[] #list of image heights and widths
    pred_class_and_score_lst = []
    gt_class_lst = []
    gt_count = 0
    for line in pred_file:
        
        l= line.split(' ')
            
        l[-1] = l[-1].strip('\n')

        name = l.pop(0)#filename

        #get prediction on correct format (list of lists)
        pred_mat=[]
        pred_class_and_score =[]
        
        if (name.split('_')[0] == vvv or vvv == None):
            for pred in l:
                p=pred.split(',')
                p_w = []
                c=0
                while(c<4):
                    p_w.append(int(round(float(p[c]))))
                    c+=1
                pred_mat.append(p_w)
        
                pc = [int(p[-1]),float(p[-2])] #class and conf score
                pred_class_and_score.append(pc)

                #get corresponding gt from filename
            #print(gt)
            gt_i = gt.get(name.replace('png','jpg')) #CHECK
                #get gt on list of lists format
            #print(gt_i)
            gt_mat=[]
            gt_class = []
            for el in gt_i['bbox']:
                gt_mat.append([int(el['xmin']),int(el['ymin']),int(el['xmax']),int(el['ymax'])]) #here is also label possible to acquire as el['label']
                if el['label'] == 'polyp':
                    gt_class.append(0)
                if el['label'] == 'Hyperplasia':
                    gt_class.append(0)
                elif el['label'] == 'Adenoma':
                    gt_class.append(1)
                gt_count = gt_count+1

            if (gt_mat != []) and (pred_mat != []): #If either the gt or the prediction contains bbox(es)
                    h_lst.append(int(gt_i['height']))
                    w_lst.append(int(gt_i['width']))
                    gt_lst.append(gt_mat)
                    gt_class_lst.append(gt_class)
                    pred_lst.append(pred_mat)
                    pred_class_and_score_lst.append(pred_class_and_score)

    return pred_lst,gt_lst,gt_class_lst,pred_class_and_score_lst,h_lst,w_lst
                



In [ ]:
def pr_curve():
    pred_lst,gt_lst,gt_class_lst,pred_class_and_score_lst,h_lst,w_lst = data_prep()
    iou_th = 0.5 #ex
    prec = []
    rec = []
    #print("GT amount: {}".format(gt_count))
    for conf_thresh in range(0,100,1):
        prec_i,rec_i,_,tp_fp_sum,tp_fn_sum = metrics(pred_lst,
                                                     gt_lst,
                                                     gt_class_lst,
                                                     pred_class_and_score_lst,
                                                     h_lst,
                                                     w_lst,
                                                     iou_th = iou_th,
                                                     thresh = conf_thresh/100)
        prec.append(prec_i)
        rec.append(rec_i)
        if conf_thresh % 10 == 0: 
            print(conf_thresh)

    return prec, rec

prec, rec = pr_curve() #takes some time

In [ ]:
#FOR TESTING
def test_pr():
    iou=0.5        
    ct = 0.22
    f1_lst = []
    for vvv in [None]:
    #for vvv in [None,'v1','v2','v3','v4','v5','v7','v8','v9','v10','v12','v13','v14','v15','v16','v18','v20','v24']:
        pred_lst,gt_lst,gt_class_lst,pred_class_and_score_lst,h_lst,w_lst = data_prep(vvv)
        prec,rec,_,_,_ = metrics(pred_lst,
                            gt_lst,
                            gt_class_lst,
                            pred_class_and_score_lst,
                            h_lst,
                            w_lst,
                            iou_th = iou,
                            thresh = ct)
        f1 = 2*prec*rec/(prec+rec+1e-16)
                
        print(f"For {vvv}, an IOUth of {iou} and conf thresh of {ct} gives f1: {f1} and precision/recall: {prec}/{rec}")
            
test_pr()

In [ ]:
from sklearn.metrics import auc
%matplotlib inline
from matplotlib import pyplot as plt

val = 22
plt.plot(rec,prec)
plt.plot(rec[val],prec[val], marker="o", markeredgecolor="red")
plt.annotate("Threshold = {}".format(val/100), (rec[val],prec[val]),color = "red")
#plt.annotate("Recall = {}".format(rec[val]), (max(rec)/2, max(prec)/2),color = "red")
#plt.annotate("Precision = {}".format(prec[val]), (max(rec)/3, max(prec)/3),color = "red")
plt.title("SNBI4 classification validation PR-Curve @.5")
plt.xlabel("Recall ({})".format(round(rec[val],3)))
plt.ylabel("Precision ({})".format(round(prec[val],3)))
plt.show()



f1 =[]
iss = []
for i in range(len(rec)):
    iss.append(i)
    f1.append(2*rec[i]*prec[i]/(rec[i]+prec[i]))

plt.plot(iss,f1)
plt.plot(iss[val],f1[val], marker="o", markeredgecolor="red")
plt.annotate("Threshold = {}".format(val/100), (iss[val],f1[val]),color = "red")
plt.title("SNBI4 classification validation F1 curve @.5")
plt.xlabel("Confidence threshold")
plt.ylabel("F1-score ({})".format(round(f1[val],3)))
plt.show()


prec2 = prec
prec2[0] = 0
rec2 = rec
rec2[-1] = 0

print(f"AUC of PR-curve: {auc(rec2,prec2)}") #yeah??